In [1]:
import pandas as pd
import pickle

import os

import psycopg2

import pyarrow.parquet as pq

from tqdm.notebook import tqdm

In [ ]:
# drop table if exists raw_data.transaction_data;
# create table raw_data.transaction_data
#     (app_id int,
#     amnt decimal(5,2),
#     currency int,
#     operation_kind int,
#     card_type int,
#     operation_type int,
#     operation_type_group int,
#     ecommerce_flag int,
#     payment_system int,
#     income_flag int,
#     mcc int,
#     country int,
#     city int,
#     mcc_category int,
#     day_of_week int,
#     hour int,
#     days_before int,
#     weekofyear int,
#     hour_diff int,
#     transaction_number int,
#     filename varchar(256),
#     loading_dt timestamp default now());
   
# drop table if exists raw_data.client_data;
# create table raw_data.client_data
#     (app_id int,
#     surname varchar(100),
#     name varchar(100),
#     fathername varchar(100),
#     sex varchar(10),
#     age int,
#     region varchar(100),
#     city varchar(100),
#     is_married bit,
#     has_children bit,
#     education varchar(50),
#     loading_dt timestamp default now());

# drop table if exists raw_data.dq;
# create table raw_data.dq (filename varchar(256), nr_rows int);

# drop table if exists raw_data.log;
# create table raw_data.log 
#     (action_type varchar(50),
#     action_dt timestamp default now(),
#     table_name varchar(256), 
#     filename varchar(256),
#     nr_rows int)

In [ ]:
conn = psycopg2.connect(dbname='kr_bruklin', user='postgres', 
                        password='admin', host='localhost', port=5432)

with conn.cursor() as curs:
    curs.execute('''
drop table if exists raw_data.transaction_data;
create table raw_data.transaction_data
    (app_id int,
    amnt decimal(5,2),
    currency int,
    operation_kind int,
    card_type int,
    operation_type int,
    operation_type_group int,
    ecommerce_flag int,
    payment_system int,
    income_flag int,
    mcc int,
    country int,
    city int,
    mcc_category int,
    day_of_week int,
    hour int,
    days_before int,
    weekofyear int,
    hour_diff int,
    transaction_number int,
    filename varchar(256),
    loading_dt timestamp default now());
   
drop table if exists raw_data.client_data;
create table raw_data.client_data
    (app_id int,
    surname varchar(100),
    name varchar(100),
    fathername varchar(100),
    sex varchar(10),
    age int,
    region varchar(100),
    city varchar(100),
    is_married bit,
    has_children bit,
    education varchar(50),
    loading_dt timestamp default now());

drop table if exists raw_data.dq;
create table raw_data.dq (filename varchar(256), nr_rows int);

drop table if exists raw_data.log;
create table raw_data.log 
    (action_type varchar(50),
    action_dt timestamp default now(),
    table_name varchar(256), 
    filename varchar(256),
    nr_rows int)''')
conn.commit()
conn.close()

In [15]:
for x in os.listdir('./raw_data')[1:2]:
    df = pq.read_table(os.path.join('./raw_data', x)).to_pandas()
    conn = psycopg2.connect(dbname='kr_bruklin', user='postgres', 
                        password='admin', host='localhost', port=5432)
    with conn.cursor() as curs:
        curs.execute(f"insert into raw_data.dq values ('{x}', {df.shape[0]})")
    conn.close()
    for i in tqdm(range(1, df.shape[0] // 1000 +2)):
#         print(f'Iteration {i} of {df.shape[0] // 1000 +1}')
        tmp = df.iloc[(i-1)*1000:i*1000].copy()
        tmp['filename'] = f"'{x}'"
        tmp['result'] = tmp.apply(lambda x: '('+','.join([str(x[c]) for c in tmp.columns])+',DEFAULT)', axis=1)
        values = ','.join(tmp.result.to_list())
        conn = psycopg2.connect(dbname='kr_bruklin', user='postgres', 
                        password='admin', host='localhost', port=5432)
        with conn.cursor() as curs:
            curs.execute(f'''insert into raw_data.transaction_data values
                            {values}''')
            curs.execute(f" insert into raw_data.log values ('insert', DEFAULT, 'raw_data.transaction_data', '{x}', {tmp.shape[0]})")
        conn.commit()  
        conn.close()